# Qwen2.5-3B-Instruct 한국어 멀티턴 대화 파인튜닝

- H100 80GB 최적화
- Flash Attention 3
- smol_koreantalk_full.jsonl 데이터셋
- LoRA + 8bit 양자화
- 데이터셋 미리 토크나이징 (멀티프로세싱 오류 방지)


In [8]:
# 환경 설정 및 임포트
import os
import sys
import logging
import time
import torch
from pathlib import Path

# 멀티프로세싱 최적 설정
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

# CUDA 메모리 최적화
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# CPU 코어 수 제한 (RAM 압박 방지 및 멀티프로세싱 안정성 향상)
# 시스템: 24 코어, 63GB RAM → 8 코어로 제한하여 안정성 확보
import multiprocessing
_original_mp_cpu_count = multiprocessing.cpu_count
_original_os_cpu_count = os.cpu_count
LIMIT_CPU_CORES = 8  # RAM 51GB 사용 가능, 안정적인 토크나이징을 위해 8개로 제한
multiprocessing.cpu_count = lambda: LIMIT_CPU_CORES
os.cpu_count = lambda: LIMIT_CPU_CORES

# psutil도 오버라이드
try:
    import psutil
    _original_psutil_cpu_count = psutil.cpu_count
    psutil.cpu_count = lambda logical=True: LIMIT_CPU_CORES
except ImportError:
    pass

# 로깅 설정
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# 모듈 경로 추가
# Jupyter notebook에서는 현재 작업 디렉토리 기준으로 경로 설정
_current_dir = Path.cwd()
# scripts 디렉토리에서 실행 중인지 확인
if _current_dir.name == 'scripts':
    _project_root = _current_dir.parent
else:
    # 상위 디렉토리에서 실행 중인 경우
    _project_root = _current_dir
_src_dir = _project_root / 'src'
if str(_src_dir) not in sys.path:
    sys.path.insert(0, str(_src_dir))

# 모듈 임포트
from qwen_finetuning_3b import Qwen3BFineTuningConfig, Qwen3BFineTuner

print("환경 설정 완료!")


: 

## 시스템 리소스 확인


In [ ]:
# GPU 확인
if not torch.cuda.is_available():
    print("\n[ ERROR ] CUDA를 사용할 수 없습니다!")
    raise RuntimeError("CUDA를 사용할 수 없습니다!")

gpu_name = torch.cuda.get_device_name(0)
gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / (1024**3)

print("="*80)
print(" 시스템 정보")
print("="*80)
print(f"GPU: {gpu_name}")
print(f"GPU 메모리: {gpu_memory_gb:.1f}GB")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"CUDA 버전: {torch.version.cuda}")

# CPU 및 RAM 정보
try:
    import psutil
    mem = psutil.virtual_memory()
    print(f"\nCPU 코어 수: {psutil.cpu_count()} (제한: {LIMIT_CPU_CORES})")
    print(f"총 RAM: {mem.total/(1024**3):.1f} GB")
    print(f"사용 가능 RAM: {mem.available/(1024**3):.1f} GB")
except:
    pass

print("="*80)


 시스템 정보
GPU: NVIDIA H100 80GB HBM3
GPU 메모리: 79.2GB
CUDA 사용 가능: True
CUDA 버전: 12.8

CPU 코어 수: 8 (제한: 8)
총 RAM: 63.0 GB
사용 가능 RAM: 34.6 GB


## 설정 생성 및 확인


In [3]:
# 설정 생성
config = Qwen3BFineTuningConfig()

# Checkpoint 재개 설정 (필요시 수정)
resume_from_checkpoint = None  # 예: "MyeongHo0621/Qwen2.5-3B-Korean" 또는 "outputs/checkpoints/checkpoint-2500"

# 설정 요약 출력
print("\n" + "="*80)
print(" 설정 요약")
print("="*80)
print(f"모델: {config.base_model}")
print(f"데이터: {config.korean_data_dir}")
print(f"  파일: {', '.join(config.data_files)}")
print(f"  최대 샘플: {config.max_samples if config.max_samples else '전체'}")
print(f"출력: {config.output_dir}")
print(f"LoRA: r={config.lora_r}, alpha={config.lora_alpha}, dropout={config.lora_dropout}")
print(f"Epoch: {config.num_train_epochs}")
print(f"배치: {config.per_device_train_batch_size} × {config.gradient_accumulation_steps} = {config.per_device_train_batch_size * config.gradient_accumulation_steps}")
print(f"학습률: {config.learning_rate}")
print(f"Max Seq: {config.max_seq_length}")
print(f"체크포인트: {config.save_steps} step마다")
if config.pre_tokenize_dataset:
    print(f"데이터셋 처리: 미리 토크나이징 (멀티프로세싱 오류 방지)")
else:
    print(f"데이터셋 처리: {config.dataset_num_proc} 프로세스 (CPU 코어 제한: 8개)")
print(f"Hub 업로드: {config.hub_model_id}")
if resume_from_checkpoint:
    print(f"\n[ INFO ] Checkpoint에서 재개: {resume_from_checkpoint}")
print("="*80 + "\n")



 설정 요약
모델: Qwen/Qwen2.5-3B-Instruct
데이터: /home/work/vss/ft_llm/data
  파일: smol_koreantalk_full.jsonl
  최대 샘플: 전체
출력: /home/work/vss/ft_llm/qwen/2.5_distil/outputs/checkpoints
LoRA: r=32, alpha=64, dropout=0.0
Epoch: 1
배치: 32 × 4 = 128
학습률: 0.0002
Max Seq: 2048
체크포인트: 500 step마다
데이터셋 처리: 미리 토크나이징 (멀티프로세싱 오류 방지)
Hub 업로드: MyeongHo0621/Qwen2.5-3B-Korean



## 파인튜너 초기화


In [4]:
# 파인튜너 생성
finetuner = Qwen3BFineTuner(config)
print("파인튜너 초기화 완료!")


파인튜너 초기화 완료!


## 1. 모델 로드


In [5]:
# 모델 로드
finetuner.load_model()

# 모델 정보 확인
print(f"\n모델: {finetuner.model}")
print(f"토크나이저: {finetuner.tokenizer}")
print(f"Vocab size: {len(finetuner.tokenizer):,}")


2025-11-18 00:51:36,114 - INFO - ================================================================================
2025-11-18 00:51:36,115 - INFO - Qwen2.5-3B-Instruct 모델 로딩
2025-11-18 00:51:36,116 - INFO - ================================================================================
2025-11-18 00:51:36,116 - INFO - 모델: Qwen/Qwen2.5-3B-Instruct
2025-11-18 00:51:36,117 - INFO - Max Seq Length: 2048
2025-11-18 00:51:36,117 - INFO - 8bit: True, 4bit: False
2025-11-18 00:51:36,117 - INFO - ================================================================================
2025-11-18 00:51:36,118 - INFO - 
2025-11-18 00:51:36,119 - INFO - [모델 로드 전] 시스템 리소스 모니터링
2025-11-18 00:51:36,119 - INFO - ================================================================================
2025-11-18 00:51:36,334 - INFO - ================================================================================
2025-11-18 00:51:36,334 - INFO - [모델 로드 전] CPU/RAM 상세 모니터링
2025-11-18 00:51:36,335 - INFO -   시간: 2025-11-18

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2025.11.3: Fast Qwen2 patching. Transformers: 4.57.1.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.19 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 9.0. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Qwen2 does not support SDPA - switching to fast eager.


Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.45s/it]
2025-11-18 00:51:57,791 - INFO - [ INFO ] ✅ Flash Attention 3 로드 성공!
2025-11-18 00:51:57,792 - INFO - [ COMPLETE ] 베이스 모델 로드 완료
2025-11-18 00:51:57,792 - INFO - 
2025-11-18 00:51:57,792 - INFO - [베이스 모델 로드 후] 시스템 리소스 모니터링
2025-11-18 00:51:57,793 - INFO - ================================================================================
2025-11-18 00:51:58,028 - INFO - ================================================================================
2025-11-18 00:51:58,029 - INFO - [베이스 모델 로드 후] CPU/RAM 상세 모니터링
2025-11-18 00:51:58,029 - INFO -   시간: 2025-11-18 00:51:58
2025-11-18 00:51:58,030 - INFO -   CPU 사용률: 29.1% (8 cores @ 2292 MHz)
2025-11-18 00:51:58,030 - INFO -   RAM: 23.6GB / 63.0GB (37.5%)
2025-11-18 00:51:58,030 - INFO -   프로세스 CPU: 0.0%
2025-11-18 00:51:58,031 - INFO -   프로세스 RAM: 1.23GB
2025-11-18 00:51:58,031 - INFO -   프로세스 스레드: 102
2025-11-18 00:51:58,035 - INFO -   Top 3 프로세스:
2025-11-18 00:51:58,03

Unsloth: Making `model.base_model.model.model` require gradients


2025-11-18 00:52:01,396 - INFO - [ COMPLETE ] LoRA 적용 완료
2025-11-18 00:52:01,397 - INFO - 
2025-11-18 00:52:01,398 - INFO - [LoRA 적용 후] 시스템 리소스 모니터링
2025-11-18 00:52:01,398 - INFO - ================================================================================
2025-11-18 00:52:01,613 - INFO - ================================================================================
2025-11-18 00:52:01,614 - INFO - [LoRA 적용 후] CPU/RAM 상세 모니터링
2025-11-18 00:52:01,614 - INFO -   시간: 2025-11-18 00:52:01
2025-11-18 00:52:01,614 - INFO -   CPU 사용률: 29.4% (8 cores @ 2292 MHz)
2025-11-18 00:52:01,615 - INFO -   RAM: 22.9GB / 63.0GB (36.3%)
2025-11-18 00:52:01,615 - INFO -   프로세스 CPU: 0.0%
2025-11-18 00:52:01,616 - INFO -   프로세스 RAM: 1.23GB
2025-11-18 00:52:01,616 - INFO -   프로세스 스레드: 102
2025-11-18 00:52:01,620 - INFO -   Top 3 프로세스:
2025-11-18 00:52:01,621 - INFO -     1. python (PID: 105596): CPU 339.2% | RAM 2.0%
2025-11-18 00:52:01,621 - INFO -     2. python (PID: 99692): CPU 100.1% | RAM 22.8%
20


모델: PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 2048, padding_idx=151654)
        (layers): ModuleList(
          (0-35): 36 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=2048, out_features=2048, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_pr

## 2. 데이터 로드


In [6]:
# 데이터 로드
dataset = finetuner.load_data()
num_samples = len(dataset)

print(f"\n데이터 로드 완료!")
print(f"총 샘플 수: {num_samples:,}")

# 샘플 확인
if len(dataset) > 0:
    print(f"\n첫 번째 샘플 키: {list(dataset[0].keys())}")
    if 'messages' in dataset[0]:
        print(f"메시지 수: {len(dataset[0]['messages'])}")
        print(f"첫 번째 메시지: {dataset[0]['messages'][0] if dataset[0]['messages'] else 'None'}")


2025-11-18 00:52:01,891 - INFO - ================================================================================
2025-11-18 00:52:01,892 - INFO - 한국어 멀티턴 대화 데이터셋 로딩
2025-11-18 00:52:01,892 - INFO - ================================================================================
2025-11-18 00:52:01,893 - INFO - 데이터 디렉토리: /home/work/vss/ft_llm/data
2025-11-18 00:52:01,893 - INFO - 
2025-11-18 00:52:01,894 - INFO - [데이터 로드 전] 시스템 리소스 모니터링
2025-11-18 00:52:01,894 - INFO - ================================================================================
2025-11-18 00:52:02,110 - INFO - ================================================================================
2025-11-18 00:52:02,111 - INFO - [데이터 로드 전] CPU/RAM 상세 모니터링
2025-11-18 00:52:02,111 - INFO -   시간: 2025-11-18 00:52:02
2025-11-18 00:52:02,112 - INFO -   CPU 사용률: 30.2% (8 cores @ 2291 MHz)
2025-11-18 00:52:02,112 - INFO -   RAM: 23.0GB / 63.0GB (36.6%)
2025-11-18 00:52:02,112 - INFO -   프로세스 CPU: 0.0%
2025-11-18 00:52:02,113 - I


데이터 로드 완료!
총 샘플 수: 460,281

첫 번째 샘플 키: ['messages', 'custom_id']
메시지 수: 6
첫 번째 메시지: {'content': '제가 편집할 내용이 있어요. 제가 작성한 텍스트는 다음과 같아요,\n\n"저와 제 친구들은 오랫동안 영화를 다시 보러 가서 우리가 기다려온 영화를 볼 날을 기다려왔어요. 어젯밤에 저는 마침내 표를 사러 갔는데, 영화관에 도착했을 때 표가 매진되었고, 그래서 저는 매우 화가 났어요. 우리 모두 화가 났어요. 그래서 오늘 우리는 대신 경기를 보러 갈지 결정했지만, 지금은 경기를 보러 가지 않을 것 같아요."', 'content_en': 'I need you to edit something for me. This is the text I wrote, \n\n"Me and my friends have been waiting for a long time to go back to the movies and catch a movie we been waiting on. Last nite I finely went to go buy tickets and when I got to the movie theater the tickets were sold out, so I was pretty pist. We were all pist. So today we deside if we wanted to go to a game instead, but I dont think we will go to a game now."', 'role': 'user'}


## 3. 데이터 포맷팅 및 토크나이징 (선택적 확인)


In [2]:
# 데이터 포맷팅 (토크나이징 포함)
# 주의: 이 단계는 시간이 오래 걸릴 수 있습니다 (460k 샘플)
# 필요시 이 셀을 건너뛰고 바로 학습을 시작할 수도 있습니다

# 작은 샘플로 먼저 테스트하려면:
# test_dataset = dataset.select(range(min(1000, len(dataset))))
# train_formatted = finetuner.format_dataset(test_dataset)

# 전체 데이터셋 포맷팅
train_formatted = finetuner.format_dataset(dataset)

print(f"\n포맷팅 완료!")
print(f"포맷팅된 샘플 수: {len(train_formatted):,}")

# 샘플 확인
if len(train_formatted) > 0:
    sample = train_formatted[0]
    print(f"\n포맷팅된 샘플 키: {list(sample.keys())}")
    if 'input_ids' in sample:
        
        print(f"input_ids 길이: {len(sample['input_ids'])}")
        # 디코딩하여 확인
        decoded = finetuner.tokenizer.decode(sample['input_ids'], skip_special_tokens=False)
        print(f"디코딩된 텍스트 (처음 200자): {decoded[:200]}...")
    elif 'text' in sample:
        print(f"텍스트 (처음 200자): {sample['text'][:200]}...")


NameError: name 'finetuner' is not defined

## 4. 학습 실행


In [ ]:
# 학습 실행
# 주의: 이 단계는 매우 오래 걸립니다 (수 시간 소요)

train_start = time.perf_counter()

try:
    # 데이터가 이미 포맷팅되어 있으면 format_dataset을 건너뛰기 위해
    # train 메서드 내부에서 다시 포맷팅하지 않도록 수정이 필요할 수 있습니다.
    # 현재는 train 메서드가 내부에서 format_dataset을 호출하므로,
    # 위에서 포맷팅한 데이터를 재사용하려면 코드 수정이 필요합니다.
    
    # 간단한 방법: 원본 dataset을 전달하고 train 메서드 내부에서 포맷팅하도록 함
    model_path = finetuner.train(dataset, resume_from_checkpoint=resume_from_checkpoint)
    
    train_end = time.perf_counter()
    total_time = train_end - train_start
    
    # 결과 출력
    total_tokens = (
        num_samples
        * config.max_seq_length
        * config.num_train_epochs
    )
    
    tokens_per_sec = total_tokens / total_time if total_time > 0 else 0
    
    print(f"\n{'='*80}")
    print(" 완료!")
    print(f"{'='*80}")
    print(f"모델: {model_path}")
    print(f"총 샘플 수: {num_samples:,}")
    print(f"총 토큰 수(대략): {total_tokens:,}")
    print(f"총 학습 시간: {total_time/3600:.2f} 시간")
    print(f"평균 처리 속도: {tokens_per_sec:,.0f} tokens/sec")
    print(f"{'='*80}\n")
    
except Exception as e:
    logger.error(f"오류 발생: {e}")
    import traceback
    traceback.print_exc()
    raise


## 5. 결과 확인 (선택적)


In [ ]:
# 학습 완료 후 결과 확인
# 체크포인트 디렉토리 확인
import os
from pathlib import Path

output_dir = Path(config.output_dir)
if output_dir.exists():
    checkpoints = sorted([d for d in output_dir.iterdir() if d.is_dir() and d.name.startswith("checkpoint-")])
    print(f"\n생성된 체크포인트:")
    for cp in checkpoints:
        print(f"  - {cp.name}")
    
    final_dir = output_dir / "final"
    if final_dir.exists():
        print(f"\n최종 모델: {final_dir}")
        files = list(final_dir.glob("*"))
        print(f"  파일 수: {len(files)}")
        if files:
            print(f"  예시 파일: {files[0].name}")
else:
    print(f"\n출력 디렉토리가 아직 생성되지 않았습니다: {output_dir}")
